In [14]:
import math
import datasets
import debug_helpers

import collections
import aer
import warnings

# pretty print variabeles on line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
# returns list with jump probabilities 
# (and special NULL jump probability)
# [-max_jump ....0 ...max_jump, NULL_jump]
def _initialize_jump_probs(sentence_pairs):
    max_jump = 0
    for (s_sentence, t_sentence) in sentence_pairs:
        s_length = len(s_sentence) - 1 #ignore NULL
        t_length = len(t_sentence)
        jump_1 = abs(1 - math.floor(s_length))
        jump_2 = abs(s_length - math.floor(s_length/t_length))
        max_jump = max(jump_1, jump_2, max_jump)
    init_prob = 1. / (2*max_jump + 1 + 1) # last item is special NULL jump prob
    return [init_prob] * (2*max_jump + 1 + 1)

# returns the index in the jump_probabilities list
# for given source and target positions and sentence lengths
# s_pos and s_length for source sentence including the special NULL word
# sentence positions start at index 0
def get_jump_prob_index(s_pos, t_pos, s_length, t_length, jump_probs):
    if s_pos == 0:
        return len(jump_probs) - 1
    jump = int(s_pos - math.floor((t_pos + 1) * (s_length - 1) / t_length))
    max_jump = int((len(jump_probs) - 2)/2)
    jump_prob_index = jump + max_jump
    if jump_prob_index < 0:
        warnings.warn(
            f'Jump prob index {jump_prob_index} (jump:{jump}) out of range.'
        )
        return 0 #approximate with prob of largest negative jump
    if jump_prob_index >= len(jump_probs) - 1:
        warnings.warn(
            f'Jump prob index {jump_prob_index} (jump:{jump}) out of range.'
        )
        return len(jump_probs) - 2 #approximate with prob of largest positive jump

    return jump_prob_index

# returns jump probability for given source and target positions
# and lengths. Positions start at index 0, source sentence contains
# special NULL word at position 0
def get_jump_prob(s_pos, t_pos, s_length, t_length, jump_probs):
    jump_prob_index = get_jump_prob_index(s_pos, t_pos, s_length, t_length, jump_probs)
    return jump_probs[jump_prob_index]

def get_alignment_prob(s_pos, t_pos, s_length, t_length, jump_probs):
    jump_prob = get_jump_prob(
        s_pos, t_pos, s_length, t_length, jump_probs)
    sum_jump_probs = sum([get_jump_prob(
        s_word_pos, t_pos, s_length, t_length, jump_probs
    ) for s_word_pos in range(s_length)])
    return jump_prob / sum_jump_probs

In [16]:
# {Hause: { book:0.25, ...}, ...}
# read: the probability of 'book' given 'Haus' is 0.25
def _initialize_lexicon_probabilities(source_vocabulary, target_vocabulary):
    p_init = 1./len(target_vocabulary)
    lexicon_probabilities = collections.defaultdict(
        lambda: collections.defaultdict(lambda: p_init))
    return lexicon_probabilities

In [17]:
def source_dependencies(s_sentence, t_word, t_pos, t_length, lprobs, jump_probs):
    s_length = len(s_sentence)
    return [
        get_alignment_prob(s_pos, t_pos, s_length, t_length, jump_probs) * lprobs[s_word][t_word] 
        for s_pos, s_word in enumerate(s_sentence)
    ]

In [18]:
def _likelihood_target_word(s_sentence, t_word, t_pos, t_length, lprobs, jump_probs):
    return sum(source_dependencies)

In [19]:
def align(lprobs, jump_probs, sentence_pairs):
    if isinstance(sentence_pairs, tuple):
        return _align_sentence_pair(lprobs, jump_probs, sentence_pairs)
    return [ _align_sentence_pair(lprobs, jump_probs, sentence_pair) for sentence_pair in sentence_pairs ]

def _align_sentence_pair(lprobs, jump_probs, sentence_pair):
    s_sentence = sentence_pair[0]
    t_sentence = sentence_pair[1]
    s_length = len(s_sentence)
    t_length = len(t_sentence)
    best_alignment = set()
    for t_pos, t_word in enumerate(t_sentence):
        best_align_prob = -1
        best_align_pos = -1
        for s_pos, s_word in enumerate(s_sentence):
            if s_word not in lprobs.keys() or t_word not in lprobs[s_word].keys():
                continue # ignore unseen source and target words
            align_prob = get_alignment_prob(s_pos, t_pos, s_length, t_length, jump_probs) * lprobs[s_word][t_word] 
            if align_prob >= best_align_prob:
                best_align_pos = s_pos
                best_align_prob = align_prob
        if (best_align_pos > 0): # Leave out NULL-alignments (and alignments between unseen words)
            best_alignment.add((best_align_pos, t_pos + 1)) # word positions start at 1
    return best_alignment


In [20]:
def EM(s_t_pairs, s_vocabulary, t_vocabulary, max_iterations = 10,
        val_sentence_pairs = None, reference_alignments = None, fn_debug = None):
    lprobs = _initialize_lexicon_probabilities(s_vocabulary, t_vocabulary)
    jump_probs = _initialize_jump_probs(s_t_pairs)
#    print('initial jump probs', jump_probs)
    i = 0
    log_likelihoods = []
    AERs = []
    while i < max_iterations:
        print(datetime.datetime.now().strftime("%I:%M"), ':', i)
        
        # initialize
        log_likelihood = 0
        AER = 0
        counts_t_given_s = collections.defaultdict(lambda: collections.defaultdict(int))
        total_s = collections.defaultdict(int)
        jump_counts = [0]*len(jump_probs)

        # calculate counts and log likelihood
        for (s_sentence, t_sentence) in s_t_pairs:
            s_length = len(s_sentence)
            t_length = len(t_sentence)
            for t_pos, t_word in enumerate(t_sentence):
                prob_counts = source_dependencies(
                    s_sentence, t_word, t_pos, t_length, lprobs, jump_probs)
                s_total_t = sum(prob_counts)
                log_likelihood += math.log(s_total_t)
                                
                for s_pos, s_word in enumerate(s_sentence):
                    update = prob_counts[s_pos]/s_total_t
                    counts_t_given_s[s_word][t_word] += update
                    total_s[s_word] += update
                    jump_count_index = get_jump_prob_index(s_pos, t_pos, s_length, t_length, jump_probs)
                    jump_counts[jump_count_index] += update 
 #                   print('jump_update', jump_count_index, '%.2f' % update, s_word, t_word)
        
        # store log_likelihood and AER values
        log_likelihoods.append(log_likelihood)
        if val_sentence_pairs and reference_alignments:
            predicted_alignments = align(lprobs, jump_probs, val_sentence_pairs)
            AER = aer.calculate_AER(reference_alignments, predicted_alignments)
            AERs.append(AER)

        # print debug info
        if fn_debug:
            fn_debug(i, lprobs, log_likelihood, AER)

        # update probabilities
        for s in lprobs.keys():
            for t in lprobs[s].keys():
                lprobs[s][t] = counts_t_given_s[s][t]/total_s[s]
        
  #      print('jump_counts', ["%.2f"%item for item in jump_counts])
        jump_count_sum = sum(jump_counts)
        jump_probs = [jc/jump_count_sum for jc in jump_counts]
    #    print(f'i:{i}, jump_probs:{["%.2f"%item for item in jump_probs]}')

        # update iteration number
        i += 1
    return lprobs, jump_probs, log_likelihoods, AERs


In [ ]:
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data_word_order()
s_t_pairs
(lprobs, jump_probs, _, _) = EM(s_t_pairs, s_vocabulary, t_vocabulary, 
                 fn_debug = debug_helpers.print_likelihood) #debug_helpers.print_likelihood debug_helpers.print_lexicon_probs

debug_helpers.print_lexicon_probs(None, lprobs, None, None)
jump_probs

In [21]:
# Run EM on toy example
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data_null_words()
s_t_pairs[0:3]
(lprobs, jump_probs, _, _) = EM(s_t_pairs[0:3], s_vocabulary, t_vocabulary, 
                 fn_debug = debug_helpers.print_likelihood)

debug_helpers.print_lexicon_probs(None, lprobs, None, None)
jump_probs

[(['<NULL>', 'das', 'Haus'], ['the', 'house']),
 (['<NULL>', 'das', 'Buch'], ['the', 'book']),
 (['<NULL>', 'ein', 'Buch'], ['a', 'book'])]

12:01 : 0
iteration  log_likelihood  likelihood  AER
0 -10.751 0.000 0.00000
12:01 : 1
1 -5.890 0.003 0.00000
12:01 : 2
2 -3.553 0.029 0.00000
12:01 : 3
3 -1.006 0.366 0.00000
12:01 : 4
4 -0.246 0.782 0.00000
12:01 : 5
5 -0.078 0.925 0.00000
12:01 : 6
6 -0.025 0.975 0.00000
12:01 : 7
7 -0.008 0.992 0.00000
12:01 : 8
8 -0.003 0.997 0.00000
12:01 : 9
9 -0.001 0.999 0.00000
<NULL> the 0.4991292185033461
<NULL> house 0.0008707814966539814
<NULL> book 0.4991292185033461
<NULL> a 0.0008707814966539814
das the 1.0
das house 5.759195849526883e-243
das book 3.467226178467145e-243
Haus the 1.7113802053279e-240
Haus house 1.0
Buch the 3.467226178467062e-243
Buch book 1.0
Buch a 5.759195849526748e-243
ein a 1.0
ein book 1.711380205327941e-240



[2.883050489433111e-241,
 0.9999246503574676,
 2.8830504894330416e-241,
 7.534964253244149e-05]

In [22]:
# Run EM on toy example with NULL words
val_sentence_pairs = [(
    ['<NULL>', 'Buch', 'klein', 'das', 'Haus'], 
    ['book', 'small', 'the', 'house']
)]
ref_alignments = [[
    {(3, 1), (2, 2), (4, 3), (1, 4)}, 
    {(3, 1), (2, 2), (4, 3), (1, 4)}
]]
s_t_pairs, s_vocabulary, t_vocabulary = datasets.example_data_null_words()
s_t_pairs
(lprobs, jump_probs, _, _) = EM(
    s_t_pairs, s_vocabulary, t_vocabulary, 20,
    val_sentence_pairs, ref_alignments, debug_helpers.print_likelihood)
debug_helpers.print_lexicon_probs(None, lprobs, None, None)
jump_probs

[(['<NULL>', 'das', 'Haus'], ['the', 'house']),
 (['<NULL>', 'das', 'Buch'], ['the', 'book']),
 (['<NULL>', 'ein', 'Buch'], ['a', 'book']),
 (['<NULL>', 'ein', 'Haus'], ['a', 'small', 'house']),
 (['<NULL>', 'mein', 'Buch'], ['my', 'small', 'book'])]

12:01 : 0
iteration  log_likelihood  likelihood  AER
0 -21.501 0.000 0.50000
12:01 : 1
1 -16.406 0.000 1.00000
12:01 : 2
2 -11.866 0.000 1.00000
12:01 : 3
3 -7.532 0.001 1.00000
12:01 : 4
4 -5.653 0.004 1.00000
12:01 : 5
5 -4.928 0.007 1.00000
12:01 : 6
6 -4.652 0.010 1.00000
12:01 : 7
7 -4.547 0.011 1.00000
12:01 : 8
8 -4.507 0.011 1.00000
12:01 : 9
9 -4.494 0.011 1.00000
12:01 : 10
10 -4.494 0.011 1.00000
12:01 : 11
11 -4.500 0.011 1.00000
12:01 : 12
12 -4.509 0.011 1.00000
12:01 : 13
13 -4.521 0.011 1.00000
12:01 : 14
14 -4.536 0.011 1.00000
12:01 : 15
15 -4.553 0.011 1.00000
12:01 : 16
16 -4.571 0.010 1.00000
12:01 : 17
17 -4.591 0.010 1.00000
12:01 : 18
18 -4.612 0.010 1.00000
12:01 : 19
19 -4.633 0.010 1.00000
<NULL> the 3.435972729711346e-13
<NULL> house 5.022798347214058e-13
<NULL> book 9.983477528928095e-10
<NULL> a 0.37979799545065424
<NULL> small 5.710624066074272e-07
<NULL> my 0.6202014324877455
das the 1.0
Haus the 7.36188380398008e-22
Haus house 0.9999999999999234
Haus sm

/home/maartje/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Jump prob index 5 (jump:3) out of range.
/home/maartje/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Jump prob index -1 (jump:-3) out of range.


[0.0, 0.0, 0.8283837307250819, 0.07382605711232869, 0.0, 0.09779021216258932]

In [ ]:
# Run EM on training data set with AER on validation set
val_sentence_pairs, _, _ = datasets.validation_data()
reference_alignments = datasets.validation_alignments()    

s_t_pairs, s_vocabulary, t_vocabulary = datasets.training_data()
(lprobs, jump_probs, log_lhoods, AERs) = EM(
    s_t_pairs, s_vocabulary, t_vocabulary, 30,
    val_sentence_pairs, reference_alignments, 
    debug_helpers.print_likelihood)

12:01 : 0
iteration  log_likelihood  likelihood  AER
0 -49501757.944 0.000 0.92877
02:16 : 1
1 -22598368.778 0.000 0.28848
04:32 : 2
2 -18009753.946 0.000 0.25570
06:49 : 3
3 -16335263.053 0.000 0.24335
09:06 : 4


In [9]:
import datetime
print(datetime.datetime.now())

2018-04-29 23:57:38.713576


In [12]:
datetime.datetime.now().strftime("%I:%M")

'11:59'